In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

<!-- ##### Object for individual customer
- Calculates net-load of each customer. Inputs: each customer's demand, microgeneration forecasts, initial SOC, battery status (0/1)
- Energy management at the household -->

In [ ]:
class Customer: # Customer Energy Management
    def __init__(self,hh_demand,pvsize,hh_SOC,batt_size, Pr_grid, fit):
    # ,Pr_grid,fit):
        self.hh_demand=hh_demand
        self.hh_gen=Pg.iloc[t].values*pvsize
        self.hh_SOC=hh_SOC
        self.batt_size=batt_size
        self.Pr_fit=fit
        self.Pr_grid=Pr_grid
        self.E=batt_size
        self.Pbmax=batt_size*Crate

        self.hh_demand = self.price_responsive_load()

        if self.batt_size>0: #battery management
            self.battery=self.battery_management()
            self.Pgrid=self.battery[0]
            self.SOC2=self.battery[1]
            self.Pb=self.battery[2]
        else: #Energy management in absence of battery
            self.Pgrid=(self.hh_demand-self.hh_gen)
            self.SOC2=0
            self.Pb=0

    def price_responsive_load(self):
        if self.Pr_grid >= Pr_th_s:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (Pr_th_s - Pr_th_b))
        elif self.Pr_grid >= Pr_th_b and self.Pr_grid <= Pr_th_s:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (self.Pr_grid - Pr_th_b))
        else:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (Pr_th_b - Pr_th_b))
        return self.hh_demand

    def battery_management(self):
        if self.hh_gen>self.hh_demand:
        # if self.Pr_fit>=Pr_th_s: #sell from the battery and the surplus gen 
            # self.Pbcal=(self.hh_SOC-SOC_min)*(self.E*nd)/dT #battery discharging
            # if self.Pbcal>=self.Pbmax:
            #     self.Pb=-self.Pbmax
            # else:
            #     self.Pb=-self.Pbcal
        # elif self.Pr_grid>Pr_th_b: #dead band
        #     self.Pb=0
        # else:
            self.Pbcal=(SOC_max-self.hh_SOC)*self.E*nc/dT #battery charging if price < buying threshold
            if self.Pbcal>=self.Pbmax:
                self.Pb=self.Pbmax
            else:
                self.Pb=self.Pbcal
        else:
        # if self.Pr_grid<=Pr_th_b:
            # self.Pbcal=(SOC_max-self.hh_SOC)*self.E/(nc*dT) #battery charging
        #     if self.Pbcal>=self.Pbmax:
        #         self.Pb=self.Pbmax
        #     else:
        #         self.Pb=self.Pbcal
        # elif self.Pr_grid<=Pr_th_s:
        #     self.Pb=0
        # else:
            self.Pbcal=(self.hh_SOC-SOC_min)*(self.E*nd)/dT #discharging
            if self.Pbcal>=self.Pbmax:
                self.Pb=-self.Pbmax
            else:
                self.Pb=-self.Pbcal
        self.Pgrid=(self.hh_demand-self.hh_gen)+self.Pb
        self.SOCend=self.hh_SOC + ((self.Pb*nc)*(dT/self.E) if self.Pb>=0 else (self.Pb/nd)*(dT/self.E))
        return self.Pgrid, self.SOCend, self.Pb

In [38]:
class CustomerOffgrid:
    def __init__(self,hh_demand,pvsize,hh_SOC,batt_size):
        self.hh_demand=hh_demand
        self.hh_gen=Pg.iloc[t].values*pvsize
        self.hh_SOC=hh_SOC
        self.batt_size=batt_size
        self.E=batt_size
        self.Pbmax=batt_size*Crate
        self.battery=self.battery_management()
        self.Pb=self.battery[0]
        self.SOC2=self.battery[1]
        self.Pcurtail=self.battery[2]
        
    def battery_management(self):
        if self.hh_gen>self.hh_demand:
            self.Pbcal=(SOC_max-self.hh_SOC)*self.E/(nc*dT) #Charging
            if self.Pbcal>=self.Pbmax:
                self.Pbreq=self.Pbmax
            else:
                self.Pbreq=self.Pbcal
            if (self.hh_gen-self.hh_demand)>=self.Pbreq:
                self.Pb=self.Pbreq 
            else:
                self.Pb=(self.hh_gen-self.hh_demand)
        else:
            self.Pbcal=(self.hh_SOC-SOC_min)*(nd*self.E)/dT
            if self.Pbcal>=self.Pbmax:
                self.Pbav=self.Pbmax
            else:
                self.Pbav=self.Pbcal
            if (self.hh_demand-self.hh_gen)>=self.Pbav:
                self.Pb=-self.Pbav
            else:
                self.Pb=-(self.hh_demand-self.hh_gen) #discharge is -ve Pb
        self.Pcurtail=self.hh_demand-self.hh_gen+self.Pb
        if self.batt_size>0:
            self.SOCend=self.hh_SOC + ((self.Pb*nc)*(dT/self.E) if self.Pb>=0 else (self.Pb/nd)*(dT/self.E))
        else:
            self.SOCend=0
        return self.Pb, self.SOCend, self.Pcurtail

In [39]:
class LCOEoff:
    def __init__(self,SystemCost):
        self.SystemCost=SystemCost
        self.lcoeoff=self.calculate_lcoeoff()
        
    def calculate_lcoeoff(self):
        self.lcoeoff=(self.SystemCost + Inv[k-1]*10/15 if k>=1 else 0 + Inv[k-2]*5/15 if k>=2 else 0)/(Pd.iloc[h,:].sum()*15)
        return self.lcoeoff

##### Object to calculate hourly bill of individual customer

In [40]:
class Bill:
    def __init__(self,demand,R_VC,R_FC,T_C,D_C,M_C,D_FC,h):
        self.demand=demand
        self.Retailer_VC=R_VC
        self.Retailer_FC=R_FC
        self.FiT=WS_price
        self.T_cost=T_C #transmission costs per kWh
        self.D_cost=D_C #distribution costs per kWh
        self.M_cost=M_C #municipal costs per kWh
        self.dist_fixed_cost_HH=D_FC
        self.electricity_charges=self.calculate_electricity_charges()
        self.energy_credits=self.calculate_energy_credits()
        self.delivery_charges=self.calculate_delivery_charges()
        self.electricity_bill=self.calculate_electricity_bill()
        

    def calculate_electricity_charges(self):
        evarcharge=np.sum(np.where(self.demand >= 0, self.demand * self.Retailer_VC, self.demand * 0))
        # evarcharge=np.sum(self.demand * self.Retailer_VC)
        # evarcharge=np.sum(np.where(self.demand >= 0, self.demand * self.Retailer_VC, 0))
        efixedcharge=len(self.demand)*self.Retailer_FC
        echarge=evarcharge+efixedcharge
        # print('evarcharge',evarcharge)
        # print(efixedcharge,'efixedcharge')
        return echarge
    
    def calculate_energy_credits(self):
        ecredit=np.sum(np.where(self.demand < 0, -self.demand * 0, 0))
        return ecredit
    
    def calculate_delivery_charges(self):
        dcharge1=[dd*(self.T_cost+self.D_cost+self.M_cost+Other_ch) if dd>0 else 0 for dd in self.demand]
        dcharge2=len(self.demand)*(self.dist_fixed_cost_HH+rate_riders)
        dcharge=(sum(dcharge1)+dcharge2)
        # print('dcharge2',dcharge2)
        return dcharge
    
    def calculate_electricity_bill(self):
        bill=(self.electricity_charges+self.Retailer_FC+self.delivery_charges)
        return bill

In [41]:
class Systemsize:
    def __init__(self,total_energy_consumption):
        self.total_energy_consumption=total_energy_consumption
        self.maxlimitpv=1
        panelsize=0.4 #kW
        # PVCost=3064.56 #$/kW
        annual_om_cost_per_kw=OnM.iloc[k*5]
        BattCost=Batteryadvanced.iloc[k*5]
        PVCost=PVadvanced.iloc[k*5]
        # BattCost=654 #$/kW

        X2=self.maxlimitpv*self.total_energy_consumption/(solar_prod_annual)
        X=X2-(pvk[k-1,h] if k>=1 else 0)-(pvk[k-2,h] if k>=2 else 0)
        X1=math.ceil(X/panelsize) #number of 0.4 kW PV panels
        # print("AEC/AEP per kW = ", X, "Number of 0.4kW Panels = ", X1)

        #PV size options: 100%, 50%, 25%, 0
        PV1=X1*panelsize 
        PV2=math.floor(X1/2)*panelsize
        PV3= math.floor(X1/4)*panelsize
        PV4=0
        data_PV = {'PVSize': [PV1, PV2, PV3, PV4],
                  'PVCost': [PVCost*PV1, PVCost*PV2, PVCost*PV3, PVCost*PV4]}
        df_PV = pd.DataFrame(data_PV)
        
        #Battery size options: round down values, closest to selected PV sizes
        B1=math.floor(PV1)/Crate
        B2=math.floor(PV2)/Crate
        B3=math.floor(PV3)/Crate
        B4=math.floor(PV4)/Crate
        # print('1:', 'PV=', PV1, "Batt=", B1, '2:', 'PV=', PV2, "Batt=",  B2, '3:', 'PV=', PV3, "Batt=", B3, '4:', 'PV=', PV4, "Batt=", B4)
        data_B = {'BattSize': [B1, B2, B3, B4],
                  'BattCost': [BattCost*B1, BattCost*B2, BattCost*B3, 0]} #price of battery
        df_B = pd.DataFrame(data_B)  
       
        self.df_combinations = pd.DataFrame(columns=['PVSize', 'BattSize', 'Total_Cost']) 

        for _, row_PV in df_PV.iterrows():
            for _, row_B in df_B.iterrows():
                total_cost = row_PV['PVCost'] + row_B['BattCost'] + annual_om_cost_per_kw*row_PV['PVSize']*systemlifetime
                data = [[row_PV['PVSize'], row_B['BattSize'], total_cost]]
                self.df_combinations = pd.concat([self.df_combinations, pd.DataFrame(data, columns=['PVSize', 'BattSize', 'Total_Cost'])])

        self.df_combinations.reset_index(drop=True, inplace=True)  # Reset the index
        print(self.df_combinations)

##### Object for individual customer's decision

In [42]:
class Decision:
    # def __init__(self,Electricity_Bill,PVSize,PVSize0,BattSize,BattSize0,BillPV0,SystemCost,BaseBilla,BaseLoada,Loff):
    def __init__(self,Electricity_Bill,PVSizedecision,BattSizedecision,SystemCost,Loff):
        self.electricity_bill=Electricity_Bill
        self.PVSizedecision=PVSizedecision
        self.BattSizedecision=BattSizedecision
        self.SystemCost=SystemCost
        self.options=options
        self.Loff=Loff
        self.LCOE=np.zeros(self.options)
        self.lcoge=self.calculate_lcoge() #levelized cost of grid electricity
        self.PV_lcoe=self.calculate_PV_lcoe() #levelized cost of each PV-batt combination
        self.system_decision=self.select_system() #logic to select one combination

    def calculate_lcoge(self): #previous investment?
        self.lcoge = round((Inv[k-2,h]*5/15 + Inv[k-1,h]*10/15 + Billexpected[k,h]*5 + Billkminus1k[h,15]*5 + Bill0[k,h]*5)/(Pd.iloc[h,:].sum()*15) if k>=2 else (Inv[k-1,h]*10/15 + Billexpected[k,h]*10 + Bill0[k,h]*5)/(Pd.iloc[h,:].sum()*15) if k==1 else (Bill0[k,h]*15)/(Pd.iloc[h,:].sum()*15),3)
        print('grid lcoe',self.lcoge)
        return self.lcoge
    
    def calculate_PV_lcoe(self):
        for i in range(0,self.options):
            self.LCOE[i]=round((Inv[k-2,h]*5/15+Inv[k-1,h]*10/15+self.SystemCost[i]+self.electricity_bill[i]*5+Billkminus1k[h,i]*5+Billonlyk[h,i]*5)/(Pd.iloc[h,:].sum()*15) if (k-2) >= 0 else (Inv[k-1,h]*10/15+self.SystemCost[i]+self.electricity_bill[i]*10+Billonlyk[h,i]*5)/(Pd.iloc[h,:].sum()*15) if k==1 else (self.SystemCost[i] + self.electricity_bill[i]*15)/(Pd.iloc[h,:].sum()*15),3)
        return(self.LCOE)
    
    def select_system(self):
        print('LCOE combinations',self.LCOE)
        self.L= np.append(self.LCOE, self.Loff)
        self.PV_add_lcoe=min(filter(lambda y: y <= self.lcoge, self.L), default=None)
        index = np.where(self.L==self.PV_add_lcoe)[0]
        self.index = index[0]
        self.Selected_PV=self.PVSizedecision[self.index]
        self.SelectedBatt=self.BattSizedecision[self.index]
        print('selected',self.L[self.index])
        print('selected PV size', self.Selected_PV)
        print('selected Batt size', self.SelectedBatt)
        return self.PV_add_lcoe

##### Object for Retailer Cost Calculation- BaU

In [43]:
#Business-as-usual
#Object for retailer energy-related and fixed-costs
class Energy_costs: 
    def __init__(self,TotalLoad1,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad,hourlynet,hourly_load,hourly_ugen,connectedhouses,dist):
        #Energy Procurement variables
        self.cont_vol=cont_vol #contract volume
        self.cont_price=cont_price #contract price
        self.total_load=TotalLoad1
        #Variable cost components
        self.ret_margin_E=ret_margin_E #energy return margin
        self.ret_margin_NE=ret_margin_NE #return margin on non-energy costs
        self.risk_margin=risk_margin
        #Fixed cost components
        self.hourlynetload=hourlynet
        self.hourly_load=hourly_load
        self.hourly_ugen=hourly_ugen
        self.no_of_HH=connectedhouses
        self.resload=HourlyResLoad
        self.dist=dist

        # self.WS_vol=self.calculate_WS_vol()
        self.energy_proc_cost=self.calculate_energy_proc_cost()
        # self.Ret_Var_Cost=self.calculate_Ret_Var_Cost()
        self.Ret_Var_Cost_pu=self.calculate_Ret_Var_Cost_pu()
        # # self.FiT=self.calculate_FiT()
        self.ret_fixed_cost=self.calculate_ret_fixed_cost()
        self.ret_fixed_cost_HH=self.calculate_ret_fixed_cost_HH()
        self.ret_ugen_payment=self.calculate_ret_ugen_payment()
        self.ret_rev=self.calculate_ret_rev()
        self.dsocharges=self.calculate_chargestoDSO()

        # self.ret_rev=self.calculate_ret_rev()
 
    def calculate_energy_proc_cost(self): #energy procurment costs from contracts and WS markets
        epcost=(WS_price*np.maximum(0,(self.hourlynetload-cont_vol))+cont_vol*cont_price).sum(axis=1)
        print('market load', np.maximum(0,(self.hourlynetload-cont_vol)).values.sum())
        print('market price', np.average(WS_price))
        print('market cost', (WS_price*np.maximum(0,(self.hourlynetload-cont_vol))).values.sum())
        print('cont cost', (cont_vol*cont_price).values.sum())               
        return epcost
    
    def calculate_Ret_Var_Cost_pu(self): #retailer's total variable costs
        print('EP cost', (self.energy_proc_cost.sum())*(1+self.ret_margin_E+self.risk_margin))
        print('load', (self.hourlynetload.values.sum()))
        self.rvc=((self.energy_proc_cost)*(1+self.ret_margin_E+self.risk_margin)/(self.hourly_load.sum(axis=1)))
        return self.rvc
    
    def calculate_ret_fixed_cost(self): #retailer's total fixed costs per hour over 15 years
        return (admin_ch)*(1+ret_margin_NE)
      
    def calculate_ret_fixed_cost_HH(self): #fixed costs per household: 
        return (self.ret_fixed_cost/(5*households.sum(axis=1)*8760)) #per hour charge
    
    def calculate_ret_ugen_payment(self): #microgeneration payment by retailer @ FiT Negative Cost
        ugenpay=0
        return ugenpay
    
    # def calculate_ret_rev(self):
    #     return ((self.Ret_Var_Cost_pu*self.hourly_load.sum(axis=1)).sum()+self.ret_fixed_cost)
    
    def calculate_ret_rev(self):
         return ((self.Ret_Var_Cost_pu*base.sum(axis=1)*1000).sum())

    def calculate_chargestoDSO(self):
        dsocharges=((self.dist)*self.resload.sum(axis=1)).sum()+Fixed_connection_charges
        return dsocharges

##### Object for Delivery Cost Calculation

In [44]:
class Delivery_Costs:
    def __init__(self,total_load,h):
        #margin included in these costs
        self.Fixed_connection_charges=Fixed_connection_charges #fixed distribution connection charges
        self.total_load=total_load
        self.no_of_HH=h
        self.T_cost=self.calculate_T_cost()
        self.D_cost=self.calculate_D_cost()
        self.M_cost=self.calculate_M_cost()
        self.dist_var_cost_pu=self.calculate_dist_var_cost_pu()
        self.dist_fixed_cost_HH=self.calculate_dist_fixed_cost_HH()

    def calculate_T_cost(self):
        return Tcost
    # /self.total_load
    
    def calculate_D_cost(self):
        return Dcost
    # /self.total_load
    
    def calculate_M_cost(self):
        return Mcost
    # /self.total_load
    
    def calculate_dist_var_cost_pu(self): #variable costs per kWh
        return (Tcost+Dcost+Mcost)
    
    def calculate_dist_fixed_cost_HH(self): #fixed costs per house
        return (self.Fixed_connection_charges/(5*households.values.sum()*8760))

In [45]:
kmax=3
houses=5
totalhouses=321900.6
yy=0
yyend=15
cagr=0.016
cagrhouses=0.02
planninghorizon=yyend-yy
hours=8760
cont_price=0.15
ret_margin_E=0.15
ret_margin_NE=0.05
risk_margin=0.05
Tcost=0.042094
Dcost=0.013033
Mcost=0.017162
# Tcost=0.042094*1.2e+10*planninghorizon
# Dcost=0.013033*1.2e+10*planninghorizon
# Mcost=0.017162*1.2e+10*planninghorizon
# Tcost=0.042094*5.5e+10*planninghorizon
# Dcost=0.013033*5.5e+10*planninghorizon
# Mcost=0.017162*5.5e+10*planninghorizon
Fixed_connection_charges=0.647903*365*planninghorizon*totalhouses*5
Other_ch=0
# 0.002252 #$/kWh balance pool allocation
rate_riders=0 #enter the values as $/h 
admin_ch=0.2392*totalhouses*5*365*planninghorizon
fees=0
SOC_min=0.10
SOC_max=1
Crate=1
Pr_th_s=0.16
Pr_th_b=0.14
dT=1
nc=0.95
nd=0.95
systemlifetime=15 #system lifetime in years
solar_prod_annual=1167   #kWh/yr
OnM=pd.read_csv('o&mcost.csv', delimiter=',', header=None)
Pg = pd.read_csv('PV.csv', delimiter=',', header=None)
contractsize = pd.read_csv('cont_vol.csv', delimiter=',', header=None)
WSprice = pd.read_csv('WSprice.csv',usecols=[1], delimiter=',', header=None)
S2 = pd.read_csv('soc.csv', delimiter=',', header=None) #5000 houses
Pd = pd.read_csv('house_load_selected.csv',delimiter=',', header=None) #5 houses
Batteryadvanced= pd.read_csv('battcost1.csv', delimiter=',', header=None)
PVadvanced=pd.read_csv('pvcost1.csv', delimiter=',', header=None)
baseload = pd.read_csv('baseload.csv',delimiter=',', header=None) #1 year

In [46]:
Retailer_VC=np.zeros((kmax+1,hours))
Retailer_FC=np.zeros(kmax)
T_cost=np.zeros(kmax)
D_cost=np.zeros(kmax)
M_cost=np.zeros(kmax)
D_Fcost=np.zeros(kmax)

DSO_charges=np.zeros((kmax))
DSO_charges_actual=np.zeros((kmax))
Retailer_VC_actual=np.zeros((kmax+1,hours))
Retailer_FC_actual=np.zeros(kmax)
Retailer_Rev=np.zeros((kmax))
Retailer_Rev_actual=np.zeros((kmax))
Retailer_Revenue_Household=np.zeros((kmax))
Retailer_Revenue_Household_actual=np.zeros((kmax))
profit=np.zeros(kmax)
actualprofit=np.zeros(kmax)

energycost=np.zeros(kmax)
energycostactual=np.zeros(kmax)

T_cost_actual=np.zeros(kmax)
D_cost_actual=np.zeros(kmax)
M_cost_actual=np.zeros(kmax)
D_Fcost_actual=np.zeros(kmax)
st=np.zeros(houses)

ugenpayment=np.zeros(kmax)
ugenpaymentactual=np.zeros(kmax)
Electricity_Bill=np.zeros((16,houses))
Electricity_Bill_actual=np.zeros((kmax,houses))
Actualbill=np.zeros((kmax,houses))
Actualbillkminus1k=np.zeros((kmax,houses))
Actualbillonlyk=np.zeros((kmax,houses))
Actualbill0=np.zeros((kmax,houses))

Bill0=np.zeros((kmax,houses))

Credits=np.zeros((kmax,houses))
Credits_actual=np.zeros((kmax,houses))
Demand=np.zeros((houses,hours))
Base_demand=np.zeros((hours))
EC2=0
EG2=0
EB2=0
hh=0
system_size_b = pd.DataFrame(0, index=range(houses), columns=[""])
batt=np.zeros(houses)

battk=np.zeros((kmax,houses))
pvk=np.zeros((kmax,houses))

s=pd.DataFrame(kmax, index=range(houses), columns=[""])
Inv=np.zeros((kmax,houses))
flags = [False] * houses

Billexpected=np.zeros((kmax,houses))

In [47]:
for k in range(0,kmax):
    print('k=', k)
    base=pd.DataFrame()
    Phour=pd.DataFrame()
    Pinj=pd.DataFrame()
    cont_vol = pd.DataFrame()
    WS_price = pd.DataFrame()
    households = pd.DataFrame()
    Phour2=pd.DataFrame()
    Pinj2=pd.DataFrame()
    
    for y in range(yy,yyend):
        ind=y-yy
        base[ind] = baseload*(1+cagr)**(y)
    
    for y in range(yy,yyend):
        ind=y-yy
        households[ind] = [totalhouses*(1+cagrhouses)**(y)]
        
    WS_price1=(WSprice[(0*hours):(hours*15)]/1000)
    WS_price = pd.DataFrame()
    values_per_column = 8760
    num_columns = 15
    for i in range(num_columns):
        start_idx = i * values_per_column
        end_idx = (i + 1) * values_per_column
        WS_price[i] = WS_price1[start_idx:end_idx].reset_index(drop=True)
         
    lcoeoffgrid = np.zeros(houses)
    PV = np.zeros(houses)
    Battery = np.zeros(houses)
    
    CustomerLoad0 = pd.DataFrame()
    CustomerLoad1 = pd.DataFrame()
    CustomerBatt1 = pd.DataFrame()
    CustomerSOC1 = pd.DataFrame()
    
    Customerloadactual = np.zeros((hours,houses))
    Customersocactual = np.zeros((hours+1,houses))
    Customerbattactual = np.zeros((hours,houses))
    CustomerLoadPVonlykactual = np.zeros((hours,houses))
    Loadkminus1kactual = np.zeros((hours,houses))

    ECurtail = np.zeros((houses,hours))
    CustomerSOCOff = np.zeros((houses,hours+1))
    CustomerbattOff = np.zeros((houses,hours))

    BillPVCombinations = np.zeros((houses,16))
    Billonlyk = np.zeros((houses,16))
    Billkminus1k = np.zeros((houses,16))
   
    Dcharge = np.zeros((houses,16))
    Echarge = np.zeros((houses,16))
    
    #EM with initial PV and Battery (Bill(k-1,k-2)) to get historic load estimate for retailer and DSO
    for h in range(0,houses):
        loads1 = []
        batp1 = []
        soc1 = []
        PV[h]=(pvk[k-2,h] if k>=2 else 0) + (pvk[k-1,h] if k>=1 else 0)
        Battery[h]=(battk[k-2,h] if k>=2 else 0) + (battk[k-1,h] if k>=1 else 0)
        SOC=SOC_max if Battery[h]>0 else 0
        soc1.append(SOC)
        for t in range(0,hours): #8760
            instance1 = Customer(Pd.iloc[h,t], PV[h], SOC, Battery[h], Retailer_VC[k,t], WS_price.iloc[t,10])
            # , Retailer_VC[k,t], WS_price.iloc[t,8])
            SOC=instance1.SOC2
            loads1.append(instance1.Pgrid)
            batp1.append(instance1.Pb)
            soc1.append(instance1.SOC2)
        column_name = f'House {h}'
        
        CustomerLoad1[column_name] = loads1
        CustomerBatt1[column_name] = batp1
    
    Load1 = np.where(CustomerLoad1 >= 0, CustomerLoad1, 0)
    Inj1 = np.where(CustomerLoad1 < 0, CustomerLoad1, 0)
    TotalLoad_h = Load1.sum() #kWh
    TotalInj_h = Inj1.sum() #kWh
    HourlyLoad1_h = (Load1.sum(axis=1)) #kWh
    HourlyInj1_h = Inj1.sum(axis=1)
    HourlyLoad0 = pd.DataFrame(HourlyLoad1_h)
    HourlyInj0 = pd.DataFrame(HourlyInj1_h)
    # HourlyLoad1 = (HourlyLoad0*321900.6)+(base*1000) #kWh
    # HourlyInj1 = HourlyInj0*321900.6
    # cont_vol=contractsize[0]*1000

    for y in range(yy,yyend):
        ind=y-yy
        cont_vol[ind]=contractsize[0]*1000 #kWh

    for y in range(yy,yyend):
        ind=y-yy
        Phour[ind] = HourlyLoad0[0]*households.iloc[0,ind]
    HourlyLoad1=(Phour)+(base*1000) #kWh
    HourlyResLoad1=(Phour)

    for y in range(yy,yyend):
        ind=y-yy
        Pinj[ind] = HourlyInj0[0]*households.iloc[0,ind]
    HourlyInj1 = (Pinj) #kWh
    
    HourlyNet1=HourlyLoad1+HourlyInj1

    TotalLoad1=(TotalLoad_h*households.values.sum())+(base.values.sum()*1000) #kWh
    print('TotalLoad_h',TotalLoad_h)
    print('households.values.sum()', households.values.sum())
    print('(base.values.sum()*1000)',(base.values.sum()*1000))
    Totalresload=(TotalLoad_h*households.values.sum()) #kWh
    TotalInj1=(TotalInj_h*households.values.sum()) #kWh
    
    TotalNet1=TotalLoad1+TotalInj1
    
    print('Reductions in houseshold load', (Pd.sum()-HourlyLoad0[0]).sum()*households.values.sum())
    a=base.sum()*1000
    print('Increased base load', (a-a[0]).sum())
    # print('Increased house load', )
    print('Total Grid Load for 15 years:', TotalLoad1, 'kWh')
    print('Total Res Load for 15 years:', Totalresload, 'kWh')
    print('Total Injection for 15 years:', TotalInj1, 'kWh')

    Distribution=Delivery_Costs(Totalresload,houses-hh)
    Retailer = Energy_costs(TotalNet1,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad1,HourlyNet1,HourlyLoad1,HourlyInj1,houses-hh,Distribution.dist_var_cost_pu)
    # Distribution=Delivery_Costs(TotalLoad1,houses-hh)
    energycost[k]=Retailer.energy_proc_cost.sum()
    Retailer_VC[k]=Retailer.Ret_Var_Cost_pu
    Retailer_FC[k]=Retailer.ret_fixed_cost_HH
    Retailer_Rev[k]=Retailer.ret_rev
    ugenpayment[k]=Retailer.ret_ugen_payment #negative value
    # profit[k]=Retailer_Rev[k]+ugenpayment[k]-energycost[k]
    T_cost[k]=Distribution.T_cost
    D_cost[k]=Distribution.D_cost
    M_cost[k]=Distribution.M_cost
    D_Fcost[k]=Distribution.dist_fixed_cost_HH
    Retailer_Rev[k]=Retailer.ret_rev
    DSO_charges[k]=Retailer.dsocharges
    print('Total Energy Cost',energycost[k] ,'C$')
    print("Retailer VC", Retailer_VC[k], 'C$/kWh')
    print("Retailer FC", Retailer_FC[k], 'C$/h')
    print("Transmission cost", T_cost[k], 'C$/kWh')
    print("Distribution cost", D_cost[k], 'C$/kWh')
    print("Municipal cost", M_cost[k], 'C$/kWh')
    print("Retailer revenue", Retailer_Rev[k], 'C$')
    print('Payment by Retailer for Injections', ugenpayment[k], 'C$')
    print('Retailer Profit', profit[k], 'C$')
    print('households',households)
    
#     # colors = px.colors.qualitative.Plotly
#     # fig = go.Figure()
#     # fig.add_traces(go.Scatter(y = HourlyLoad1, mode = 'lines', line=dict(color=colors[0]), name='Load'))
#     # fig.add_traces(go.Scatter(y = Retailer_VC[k], mode = 'lines', line=dict(color=colors[0]), name='Retail Price'))
#     # fig.update_layout(title='Base')
#     # # pio.write_html(fig, 'Offgrid.html')
#     # fig.show()
    
    for h in range(0,houses):
        billexp = Bill(CustomerLoad1.iloc[:,h],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
        Billexpected[k,h] = billexp.electricity_bill
    Retailer_Revenue_Household[k]=Billexpected[k].sum()*(households.sum(axis=1))
    profit[k]=Retailer_Revenue_Household[k]+Retailer_Rev[k]-energycost[k]-DSO_charges[k]+ugenpayment[k]-admin_ch
    print("Retailer revenue total", Retailer_Revenue_Household[k]+Retailer_Rev[k], 'C$')
    print('Cost to DSO', DSO_charges[k], 'C$')
    print('Retailer Profit', profit[k], 'C$')

    
    
    for h in range(0,houses):
        billexp = Bill(CustomerLoad1.iloc[:,h],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
        Billexpected[k,h] = billexp.electricity_bill
        loads0 = []
        SOC0=0
        batt0=0
        for t in range(0,hours):
            Generation0=0
            instance0 = Customer(Pd.iloc[h,t],Generation0, SOC0, batt0, Retailer_VC[k,t], WS_price.iloc[t,10])
            # , Retailer_VC[k,t], WS_price.iloc[t,8])
            loads0.append(instance0.Pgrid)
        column_name = f'House {h}'
        CustomerLoad0[column_name] = loads0
        bill0 = Bill(CustomerLoad0.iloc[:,h],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
        Bill0[k,h] = bill0.electricity_bill

        systemsize=Systemsize(Pd.iloc[h,:].sum())
        options=len(systemsize.df_combinations)
        
        #OFFGRID OPERATION
        SOC=SOC_max #fully charged battery
        PVSizeOff=systemsize.df_combinations.iloc[0,0]+PV[h]
        BattSizeOff=systemsize.df_combinations.iloc[0,1]+Battery[h]
        SystemCostOff=systemsize.df_combinations.iloc[0,2]
        genoff=[]
        curtailoff=[]
        battoff=[]
        CustomerSOCOff[h,0]=SOC_max if BattSizeOff>0 else 0
        SOC=CustomerSOCOff[h,0]
        for t in range(0,hours):
            instanceOff = CustomerOffgrid(Pd.iloc[h,t], PVSizeOff, SOC, BattSizeOff)
            SOC=instanceOff.SOC2 
            CustomerSOCOff[h,t+1]=SOC
            ECurtail[h,t]=instanceOff.Pcurtail
            CustomerbattOff[h,t]=instanceOff.Pb 

        if all(x <= 0 for x in ECurtail[h]):
            loff=LCOEoff(SystemCostOff)
            lcoeoffgrid[h]=loff.lcoeoff
        else:
            lcoeoffgrid[h]=100  
 
        # PV-Batt Combinations
        #ONLY new options at k, no previous PV-Batt
        if flags[h]==False:
            CustomerLoadPVonlyk=np.zeros((hours,options))
            CustomerSOCPVonlyk=np.zeros((hours+1,options))
            CustomerBattPVonlyk=np.zeros((hours,options))
            PVSizeonlyk=np.zeros(options)
            BattSizeonlyk=np.zeros(options)
            SystemCost=np.zeros(options)
            for i in range (0,options):
                PVSizeonlyk[i]=systemsize.df_combinations.iloc[i,0]
                BattSizeonlyk[i]=systemsize.df_combinations.iloc[i,1]
                SystemCost[i]=systemsize.df_combinations.iloc[i,2]
                load3=[]
                gen3=[]
                soc3=[]
                batt3=[]
                CustomerSOCPVonlyk[0,i]=SOC_max if BattSizeonlyk[i]>0 else 0
                SOC=CustomerSOCPVonlyk[0,i]
                for t in range(0,hours):
                    instance3 = Customer(Pd.iloc[h,t], PVSizeonlyk[i], SOC, BattSizeonlyk[i], Retailer_VC[k,t], WS_price.iloc[t,10])
                    # , Retailer_VC[k,t], WS_price.iloc[t,8])
                    SOC=instance3.SOC2
                    CustomerLoadPVonlyk[t,i]=instance3.Pgrid
                    CustomerSOCPVonlyk[t+1,i]=instance3.SOC2
                    CustomerBattPVonlyk[t,i]=instance3.Pb
                billonlyk=Bill(CustomerLoadPVonlyk[:,i],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
                Billonlyk[h,i]=billonlyk.electricity_bill

            # PREVIOUS PV-BATT at k-1 and new options at k
            Loadkminus1k=np.zeros((hours,options))
            SOCkminus1k=np.zeros((hours+1,options))
            Battkminus1k=np.zeros((hours,options))
            PVSizekminus1k=np.zeros(options)
            BattSizekminus1k=np.zeros(options)
            for i in range (0,options):
                PVSizekminus1k[i]=(systemsize.df_combinations.iloc[i,0]) + (pvk[k-1,h] if k>=1 else 0)
                BattSizekminus1k[i]=(systemsize.df_combinations.iloc[i,1]) + (battk[k-1,h] if k>=1 else 0)
                load4=[]
                gen4=[]
                soc4=[]
                batt4=[]
                SOCkminus1k[0,i]=SOC_max if BattSizekminus1k[i]>0 else 0
                SOC=SOCkminus1k[0,i]
                for t in range(0,hours):
                    instance4 = Customer(Pd.iloc[h,t], PVSizekminus1k[i], SOC, BattSizekminus1k[i], Retailer_VC[k,t], WS_price.iloc[t,10])
                    # , Retailer_VC[k,t], WS_price.iloc[t,8])
                    SOC=instance4.SOC2
                    Loadkminus1k[t,i]=instance4.Pgrid
                    SOCkminus1k[t+1,i]=instance4.SOC2
                    Battkminus1k[t,i]=instance4.Pb
                billkminus1k=Bill(Loadkminus1k[:,i],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
                Billkminus1k[h,i]=billkminus1k.electricity_bill

            # PREVIOUS PV-BATT at k-1, k-2, and new options at k
            CustomerLoadPV=np.zeros((hours,options))
            CustomerSOCPV=np.zeros((hours+1,options))
            CustomerBattPV=np.zeros((hours,options))
            PVSize=np.zeros(options)
            BattSize=np.zeros(options)
            for i in range (0,options):
                PVSize[i]=systemsize.df_combinations.iloc[i,0]+PV[h]
                BattSize[i]=systemsize.df_combinations.iloc[i,1]+Battery[h]
                load2=[]
                gen2=[]
                soc2=[]
                batt2=[]
                CustomerSOCPV[0,i]=SOC_max if BattSize[i]>0 else 0
                SOC=CustomerSOCPV[0,i]
                for t in range(0,hours):
                    instance2 = Customer(Pd.iloc[h,t], PVSize[i], SOC, BattSize[i], Retailer_VC[k,t], WS_price.iloc[t,10])
                    # , Retailer_VC[k,t], WS_price.iloc[t,8])
                    SOC=instance2.SOC2
                    CustomerLoadPV[t,i]=instance2.Pgrid
                    CustomerSOCPV[t+1,i]=instance2.SOC2
                    CustomerBattPV[t,i]=instance2.Pb
                bill=Bill(CustomerLoadPV[:,i],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
                BillPVCombinations[h,i]=bill.electricity_bill
                Echarge[h,i]=bill.electricity_charges
                Dcharge[h,i]=bill.delivery_charges

            PVSizedecision=np.append(systemsize.df_combinations.iloc[:,0].values,PVSizeOff)
            BattSizedecision=np.append(systemsize.df_combinations.iloc[:,1], BattSizeOff)
            SystemCost=np.append(SystemCost,SystemCostOff)

            decision=Decision(BillPVCombinations[h],PVSizedecision,BattSizedecision,SystemCost,lcoeoffgrid[h]) #change name to lcoe

            ind=decision.index
            battk[k,h]=decision.SelectedBatt
            pvk[k,h]=decision.Selected_PV
            Inv[k,h]=SystemCost[ind]

            if ind==options:
                hh=hh+1
                flags[h] = True  # Set the flag to True for the houses that go off-grid
                Customerloadactual[:,h]==np.zeros(hours)
            else:
                Customerloadactual[:,h]=CustomerLoadPV[:,ind]
            Customerbattactual[:,h]=CustomerBattPV[:,ind]
            Customersocactual[:,h]=CustomerSOCPV[:,ind]
            Loadkminus1kactual[:,h]=Loadkminus1k[:,ind]
            CustomerLoadPVonlykactual[:,h]=CustomerLoadPVonlyk[:,ind]
            # Billexpected[k,h]=Billkminus1k[h,ind]

    Load2 = np.where(Customerloadactual >= 0, Customerloadactual, 0)
    Inj2 = np.where(Customerloadactual < 0, Customerloadactual, 0)
    TotalLoad2_h = Load2.sum() #kWh
    TotalInj2_h = Inj2.sum() #kWh
    HourlyLoad2_h = (Load2.sum(axis=1)) #kWh
    HourlyInj2_h = (Inj2.sum(axis=1)) #kWh
    HourlyLoad20 = pd.DataFrame(HourlyLoad2_h)
    HourlyInj20 = pd.DataFrame(HourlyInj2_h)
    
    for y in range(yy,yyend):
        ind=y-yy
        Phour2[ind] = HourlyLoad20[0]*households.iloc[0,ind]
    HourlyLoad2=(Phour2)+(base*1000) #kWh
    HourlyResLoad2=(Phour2)

    for y in range(yy,yyend):
        ind=y-yy
        Pinj2[ind] = HourlyInj20[0]*households.iloc[0,ind]
    HourlyInj2 = (Pinj2) #kWh
    
    HourlyNet2=HourlyLoad2+HourlyInj2
    
    TotalLoad2=(TotalLoad2_h*households.values.sum())+(base.values.sum()*1000) #kWh
    Totalresload2=(TotalLoad2_h*households.values.sum()) #kWh
    TotalInj2=(TotalInj2_h*households.values.sum()) #kWh
    
    TotalNet2=TotalLoad2+TotalInj2
    
    print('Actual Annual Load:',TotalLoad2, 'kWh')
    print('Actual Res Load:',Totalresload2, 'kWh')
    print('Actual Annual Injection:', TotalInj2, 'kWh')
    
    Distributionactual=Delivery_Costs(Totalresload2,houses-hh)
    Retaileractual=Energy_costs(TotalNet2,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad2,HourlyNet2,HourlyLoad2,HourlyInj2,houses-hh,Distributionactual.dist_var_cost_pu)
    # Distributionactual=Delivery_Costs(TotalLoad2,houses-hh)
    energycostactual[k]=Retaileractual.energy_proc_cost.sum()
    Retailer_VC_actual[k]=Retaileractual.Ret_Var_Cost_pu
    Retailer_FC_actual[k]=Retaileractual.ret_fixed_cost_HH
    # Retailer_Rev_actual[k]=Retaileractual.ret_rev
    ugenpaymentactual[k]=Retaileractual.ret_ugen_payment
    # actualprofit[k]=Retailer_Rev_actual[k]+ugenpaymentactual[k]-energycostactual[k]
    Retailer_Rev_actual[k]=Retaileractual.ret_rev
    DSO_charges_actual[k]=Retaileractual.dsocharges
    Retailer_VC[k+1]=Retaileractual.Ret_Var_Cost_pu
    T_cost_actual[k]=Distributionactual.T_cost
    D_cost_actual[k]=Distributionactual.D_cost
    M_cost_actual[k]=Distributionactual.M_cost
    D_Fcost_actual[k]=Distributionactual.dist_fixed_cost_HH
    print('Actual Total Energy Cost',energycostactual[k] ,'C$')
    print("Actual Retailer VC", Retailer_VC_actual[k], 'C$/kWh')
    print("Actual Retailer FC", Retailer_FC_actual[k], 'C$/h')
    print("Actual Transmission cost", T_cost_actual[k], 'C$/kWh')
    print("Actual Distribution cost", D_cost_actual[k], 'C$/kWh')
    print("Actual Municipal cost", M_cost_actual[k], 'C$/kWh')
    print('Actual Retailer Revenue', Retailer_Rev_actual[k], 'C$')
    print('Actual Payment by Retailer for Injections', ugenpaymentactual[k], 'C$')
    print('Actual Retailer Profit', actualprofit[k], 'C$')
    
#     fig = go.Figure()
#     fig.add_traces(go.Scatter(y = HourlyLoad2, mode = 'lines', line=dict(color=colors[0]), name='Load'))
#     fig.add_traces(go.Scatter(y = Retailer_VC_actual[k], mode = 'lines', line=dict(color=colors[1]), name='Retail Price'))
#     fig.update_layout(title='Actual')
    
#         # pio.write_html(fig, 'Offgrid.html')
    
# BILL WITH investment at k-2,k-1,k
    for h in range(0,houses):
        if flags[h]==False:
            bill_actual=Bill(Customerloadactual[:,h],Retailer_VC_actual[k],Retailer_FC_actual[k],T_cost_actual[k],D_cost_actual[k],M_cost_actual[k],D_Fcost_actual[k],h)
            Actualbill[k,h]=bill_actual.electricity_bill #total bill of each house

            bill_actual_kminus1k=Bill(Loadkminus1kactual[:,h],Retailer_VC_actual[k],Retailer_FC_actual[k],T_cost_actual[k],D_cost_actual[k],M_cost_actual[k],D_Fcost_actual[k],h)
            Actualbillkminus1k[k,h]=bill_actual_kminus1k.electricity_bill

            bill_actual_onlyk=Bill(CustomerLoadPVonlykactual[:,h],Retailer_VC_actual[k],Retailer_FC_actual[k],T_cost_actual[k],D_cost_actual[k],M_cost_actual[k],D_Fcost_actual[k],h)
            Actualbillonlyk[k,h]=bill_actual_onlyk.electricity_bill       

            bill0actual = Bill(CustomerLoad0.iloc[:,h],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
            Actualbill0[k,h] = bill0actual.electricity_bill
            
    Retailer_Revenue_Household_actual[k]=Actualbill[k].sum()*(households.sum(axis=1))
    actualprofit[k]=Retailer_Revenue_Household_actual[k]+Retailer_Rev_actual[k]-energycostactual[k]-DSO_charges_actual[k]+ugenpaymentactual[k]-admin_ch
    print("Retailer revenue total", Retailer_Revenue_Household_actual[k]+Retailer_Rev_actual[k], 'C$')
    print('Cost to DSO', DSO_charges_actual[k], 'C$')
    print('Retailer Profit', actualprofit[k], 'C$')
            
    dfpv = pd.DataFrame({f'PV{j+1}': pvk[:, j] for j in range(pvk.shape[1])})
    dfb = pd.DataFrame({f'Batt{j+1}': battk[:, j] for j in range(battk.shape[1])})
    print(dfpv)
    print(dfb)
    print("-------------------------------")
    yy=yy+5
    yyend=yyend+5

k= 0
TotalLoad_h [37714.60083216]
households.values.sum() 5566761.281377016
(base.values.sum()*1000) 669202285712.9642
Reductions in houseshold load [0.]
Increased base load 71781081763.25415
Total Grid Load for 15 years: [8.79150465e+11] kWh
Total Res Load for 15 years: [2.0994818e+11] kWh
Total Injection for 15 years: 0.0 kWh
market load [2.55529271e+11]
market price 0.06597014216133942
market cost [1.7442721e+10]
cont cost 93543179220.0
EP cost [1.3318308e+11]
load [8.79150465e+11]


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2844\4275954818.py:127: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Retailer_FC[k]=Retailer.ret_fixed_cost_HH


Total Energy Cost 110985900203.14035 C$
Retailer VC [0.14180014 0.13919325 0.13723829 ... 0.14062304 0.14044411 0.14089675] C$/kWh
Retailer FC 0.009077153506482069 C$/h
Transmission cost 0.042094 C$/kWh
Distribution cost 0.013033 C$/kWh
Municipal cost 0.017162 C$/kWh
Retailer revenue 100992553721.69186 C$
Payment by Retailer for Injections 0.0 C$
Retailer Profit 0.0 C$
households          0           1             2              3              4   \
0  321900.6  328338.612  334905.38424  341603.491925  348435.561763   

              5              6              7             8              9   \
0  355404.272999  362512.358459  369762.605628  377157.85774  384701.014895   

              10             11             12             13             14  
0  392395.035193  400242.935897  408247.794615  416412.750507  424741.005517  


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2844\4275954818.py:159: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Retailer_Revenue_Household[k]=Billexpected[k].sum()*(households.sum(axis=1))


Retailer revenue total 156282846015.98492 C$
Cost to DSO 20886283935.679928 C$
Retailer Profit 22302824558.304646 C$
    PVSize  BattSize                                 Total_Cost
0      3.2       3.0  0    13482.603438
Name: 0, dtype: float64
1      3.2       1.0  0    12238.480308
Name: 0, dtype: float64
2      3.2       0.0  0    11616.418743
Name: 0, dtype: float64
3      3.2       0.0  0    11616.418743
Name: 0, dtype: float64
4      1.6       3.0   0    7674.394066
Name: 0, dtype: float64
5      1.6       1.0   0    6430.270937
Name: 0, dtype: float64
6      1.6       0.0   0    5808.209372
Name: 0, dtype: float64
7      1.6       0.0   0    5808.209372
Name: 0, dtype: float64
8      0.8       3.0    0    4770.28938
Name: 0, dtype: float64
9      0.8       1.0   0    3526.166251
Name: 0, dtype: float64
10     0.8       0.0   0    2904.104686
Name: 0, dtype: float64
11     0.8       0.0   0    2904.104686
Name: 0, dtype: float64
12     0.0       3.0   0    1866.184695
Name: 0, dt

...

C:\Users\ASUS\AppData\Local\Temp\ipykernel_2844\4275954818.py:221: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  SystemCost[i]=systemsize.df_combinations.iloc[i,2]


grid lcoe 0.251
LCOE combinations [0.282 0.265 0.255 0.255 0.266 0.251 0.241 0.241 0.266 0.254 0.245 0.245
 0.268 0.259 0.251 0.251]
selected 0.241
selected PV size 1.2000000000000002
selected Batt size 0.0
Actual Annual Load: 946419520495.0968 kWh
Actual Res Load: 162097343282.34912 kWh
Actual Annual Injection: -73826771247.52876 kWh
market load 282724232764.94995
market price 0.06597014216133942
market cost 16403757492.70625
cont cost 93543179220.0
EP cost 131936324055.2475
load 872592749247.568
Actual Total Energy Cost 109946936712.70625 C$
Actual Retailer VC [0.1272417  0.12370524 0.12101923 ... 0.12578811 0.12560807 0.12627862] C$/kWh
Actual Retailer FC 0.00744642744674837 C$/h
Actual Transmission cost 0.042094 C$/kWh
Actual Distribution cost 0.013033 C$/kWh
Actual Municipal cost 0.017162 C$/kWh
Actual Retailer Revenue 109256881249.07362 C$
Actual Payment by Retailer for Injections 0.0 C$
Actual Retailer Profit 0.0 C$


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2844\4275954818.py:350: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Retailer_FC_actual[k]=Retaileractual.ret_fixed_cost_HH


Retailer revenue total 151577000716.91595 C$
Cost to DSO 17427194825.13201 C$
Retailer Profit 22095031860.21769 C$
   PV1  PV2  PV3  PV4  PV5
0  0.8  1.6  0.0  2.4  1.2
1  0.4  1.2  2.0  2.0  0.8
2  0.8  2.0  1.2  2.8  1.2
   Batt1  Batt2  Batt3  Batt4  Batt5
0    0.0    0.0    0.0    0.0    0.0
1    0.0    0.0    0.0    0.0    0.0
2    0.0    0.0    0.0    0.0    0.0
-------------------------------


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2844\4275954818.py:393: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Retailer_Revenue_Household_actual[k]=Actualbill[k].sum()*(households.sum(axis=1))


In [48]:
Actualbill.sum(axis=0)*5/(15*12)

array([ 64.1155442 , 114.08223483, 159.76704849, 171.19207579,
        84.23573757])

In [49]:
actualprofit/(Retailer_Revenue_Household_actual+Retailer_Rev_actual)

array([0.14486067, 0.14565973, 0.14576771])

In [50]:
Actualbill

array([[ 852.34691558, 1546.12256094, 2312.3305149 , 2328.74248586,
        1143.92687205],
       [ 767.28689883, 1352.23126768, 1803.41093572, 2019.72627679,
         999.4867525 ],
       [ 688.52577681, 1208.6066251 , 1635.87229518, 1814.44596585,
         889.07292792]])

In [51]:
Retailer.ret_fixed_cost/(5*households.sum(axis=1))

0    65.230704
dtype: float64

In [52]:
Fixed_connection_charges

5709339976.594275

In [53]:
((Distribution.dist_var_cost_pu)*Retaileractual.resload.sum(axis=1)).sum()

11717854848.537737

In [54]:
DSO_charges_actual

array([1.77809435e+10, 1.72170261e+10, 1.74271948e+10])

In [55]:
# # simple_array = [element[0] for element in HourlyLoad1]
# import datetime
# colors = px.colors.qualitative.Plotly
# fig = go.Figure()
# # # start_date = datetime.datetime(2012, 1, 1, 23)
# # # end_date = datetime.datetime(2022, 12, 31, 23)
# # # date_rng = pd.date_range(start=start_date, end=end_date, freq='H')
# # # timestamps = [datetime.datetime(2000, 1, 1, i) for i in range(0,23) ] * 365 * 20  # Repeated for 20 years

# # fig.add_traces(go.Scatter(y = HourlyLoad2[0]*(0.0000001), mode = 'lines', line=dict(color=colors[3]), name='Load'))
# # fig.add_traces(go.Scatter(y = Retaileractual.energy_proc_cost.astype(int)/10000000, mode = 'lines', line=dict(color=colors[4]), name='cost'))
# # fig.add_traces(go.Scatter(y = WS_price/1000, mode = 'lines', line=dict(color=colors[2]), name='MSP'))
# fig.add_traces(go.Scatter(y = Retailer_VC_actual[1], mode = 'lines', line=dict(color=colors[1]), name='Retail Price Actual k = 0'))
# fig.add_traces(go.Scatter(y = Retailer_VC[1], mode = 'lines', line=dict(color=colors[0]), name='Retail Price Base'))
# # fig.add_traces(go.Scatter(y = Retailer_VC_actual[1], mode = 'lines', line=dict(color=colors[2]), name='Retail Price Actual k = 1'))
# # fig.add_traces(go.Scatter(y = Retailer_VC_actual[2], mode = 'lines', line=dict(color=colors[3]), name='Retail Price Actual k = 2'))

# # fig.add_traces(go.Scatter(y = Retailer_VC_actual[0], mode = 'lines', line=dict(color=colors[0]), name='Retail Price Actual'))

# # fig.update_layout(title='Market Settlement Price ($/MWh)')
# # fig.update_xaxes(title_text='Date', tickformat='%Y-%m-%d')
# # fig.update_yaxes(title_text="Price") # Y-axis label
# # # pio.write_html(fig, 'Prices2.html')
# # # fig.show()
              


In [56]:
Actualbill.sum(axis=0)*5/(15*12)

array([ 64.1155442 , 114.08223483, 159.76704849, 171.19207579,
        84.23573757])

In [57]:
Billexpected

array([[1073.85008594, 1995.34509106, 2371.71984306, 3009.73124495,
        1481.57275779],
       [ 806.4379338 , 1476.7687196 , 2223.49077619, 2234.54444195,
        1088.94285536],
       [ 728.18025141, 1296.11721937, 1736.27572139, 1945.97657345,
         953.87709158]])

In [58]:
MonthInv=np.zeros((1,5))
MonthInv2=Inv
for h in range(0,5):
    MonthInv[0,h]=(Inv[0,h]+Inv[1,h]+Inv[2,h])/(15*12)
print(MonthInv)

[[28.17463833 64.83386004 35.31063686 97.93694172 44.72617917]]


In [59]:
np.average(Retailer_VC_actual[0])

0.15286306822629014

In [60]:
np.average(Retailer_VC_actual[1])

0.14683296541936067

In [61]:
np.average(Retailer_VC_actual[2])

0.13939280846069713

In [62]:
TotalLoad_h

array([25818.48957105])

In [63]:
(HourlyResLoad1.sum(axis=1)*(Distribution.T_cost+Distribution.D_cost+Distribution.M_cost+Retailer_VC[0])).sum()

array([3.90024577e+10])

In [64]:
HourlyResLoad1.sum(axis=1)

0        [30484738.78462761]
1        [27954037.23590537]
2       [27731224.139243808]
3       [27935793.021002665]
4       [28592412.349385098]
                ...         
8755     [51697233.24729967]
8756     [49223444.67112163]
8757      [45018129.2668183]
8758     [36958595.28468447]
8759     [30035115.33830766]
Length: 8760, dtype: object

In [65]:
Distributionactual.dist_var_cost_pu

0.07228899999999999

In [66]:
Fixed_connection_charges/(5*8760*households.sum(axis=1))

0    0.019209
dtype: float64

In [67]:
Retailer.ret_fixed_cost/(5*8760*households.sum(axis=1))

0    0.007446
dtype: float64

In [68]:
2213229184.803+5709339976.594275+47011543528.26515

54934112689.66243

In [69]:
csv_file_path = 'O3.csv'  # Specify the file path where you want to save the CSV file
a.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV file

In [70]:
data=(WSprice[(yy*hours):(hours*yyend)][1]/1000)

df = pd.DataFrame(data)

# Number of values per new column
values_per_column = 8760

# Calculate the number of new columns needed
num_new_columns = len(df) // values_per_column

# Reshape the data using NumPy
new_data = np.array_split(df, num_new_columns)

# Create a new DataFrame with the reshaped data
new_df = pd.DataFrame(new_data)

# Transpose the new DataFrame to have values in separate columns
new_df = new_df.T

# Rename the columns if needed (optional)
new_df.columns = [f'Column{i + 1}' for i in range(num_new_columns)]

# Now, new_df contains the values split into multiple columns


ValueError: Must pass 2-d input. shape=(10, 8760, 1)

In [ ]:
import pandas as pd
import numpy as np

# Sample 3D array with shape (15, 8760, 1)
# data_3d = np.random.rand(15, 8760, 1)  # Replace this with your actual data

# Reshape into a 2D array with shape (15, 8760)
data_2d = new_data.reshape(15, -1)

# Create a DataFrame
df = pd.DataFrame(data_2d)

# Optionally, you can set column names if needed
column_names = [f'Column{i + 1}' for i in range(df.shape[1])]
df.columns = column_names
print(df.T)


In [ ]:
# csv_file = 'output1.csv'

# # Write the array into the CSV file
# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(a)

# print("Array written to", csv_file)

In [ ]:
def calculate_ret_fixed_cost(self): #retailer's total fixed costs per hour over 15 years
        return (admin_ch)*(1+ret_margin_NE)
      
    def calculate_ret_fixed_cost_HH(self): #fixed costs per household: 
        return np.average(self.ret_fixed_cost/(households*5*8760)) #per hour charge


In [ ]:
(admin_ch)*(1+ret_margin_NE)/(5*households.sum(axis=1))

In [ ]:
np.average(WS_price)

In [ ]:
(households*5*37714.60083216303).sum(axis=1)

In [ ]:
self.energy_proc_cost)*(1+self.ret_margin_E+self.risk_margin)/(self.hourly_load.sum(axis=1))

In [ ]:
Retaileractual.energy_proc_cost.sum()

In [ ]:
Retaileractual.hourly_load.sum(axis=1).sum()